In [2]:
#import all required libs not much are required so
import pandas as pd
import math
print('libs imported')

libs imported


In [233]:
def getdata():
    data = {
            'outlook':['sunny','sunny','overcast','rainy','rainy','rainy','overcast','sunny','sunny','rainy','sunny','overcast','overcast','rainy'],
            'temp':['hot','hot','hot','mild','cool','cool','cool','mild','cool','mild','mild','mild','hot','mild'],
            'humidity':['high','high','high','high','normal','normal','normal','high','normal','normal','normal','high','normal','high'],
            'windy':[False,True,False,False,False,True,True,False,False,False,True,True,False,True],
            'play':['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']
    }
    data_frame = pd.DataFrame(data)
    return data_frame

def getBankData(split_fraction):
    if(split_fraction>1 and split_fraction<=0):
        print('Invalid Split Fraction')
        return None

    #read the data
    data_frame = pd.read_csv('bank.csv');
    
    
    data_records = data_frame[list(data_frame.columns)].to_dict(orient = 'records')
    new_data = []
    
    #PROCESSING OF DATA
    for data_record in data_records:
        
        #ASSIGN CLASS TO BALANCE RANGES
        if(data_record['balance']<10000):
            data_record['balance'] = 'LessThan10000'
        elif(data_record['balance']>=10000 and data_record['balance']<20000) :
            data_record['balance'] = 'LessThan20000'
        elif(data_record['balance']>=20000 and data_record['balance']<30000) :
            data_record['balance'] = 'LessThan30000'
        elif(data_record['balance']>=30000 and data_record['balance']<40000) :
            data_record['balance'] = 'LessThan40000'
        elif(data_record['balance']>=40000 and data_record['balance']<50000) :
            data_record['balance'] = 'LessThan50000'
        else:
            data_record['balance'] = 'GreaterThan500000'
            
        #ASSIGN CLASS DURATION RANGES
        if(data_record['duration']<=365):
            data_record['duration']='LessThan365secs'
        elif(data_record['duration']>365 and data_record['duration']<=730):
            data_record['duration']='LessThan730secs'
        elif(data_record['duration']>730 and data_record['duration']<=995):
            data_record['duration']='LessThan995secs'
        else:
            data_record['duration']='MoreThan995secs'
        
        #ASSIGN CLASS AGE RANGES
        if(data_record['age']<=40):
            data_record['age']='Adults'
        elif(data_record['age']>40 and data_record['age']<=60):
            data_record['age']='MiddleAged'
        else:
            data_record['age']='SeniorCitizens'
        
        new_data.append(data_record);
        
    data_frameq = pd.DataFrame(new_data);
    del data_frameq['pdays']
    #randomize the order of data
    data_frameq = data_frameq.sample(frac=1).reset_index(drop=True)
    
    #find the split point
    mid = int(round(split_fraction*len(data_frameq.index),0))
    
    #split the data
    train_data = data_frameq.iloc[ : mid , : ]
    test_data = data_frameq.iloc[ mid : , :  ]

    #return 
    return train_data,test_data

In [234]:
#test getdata and getBankData
data = getdata()
data.head()

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [235]:
#train_data
train_data,test_data=getBankData(0.5)
# train_data.head()
# train_data.describe()
print(sorted(train_data['day'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [236]:
#test_data
test_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,deposit
5581,Adults,management,married,tertiary,no,LessThan10000,yes,no,telephone,21,jul,LessThan995secs,3,0,unknown,yes
5582,MiddleAged,services,married,primary,no,LessThan10000,yes,no,cellular,15,feb,LessThan365secs,1,0,unknown,yes
5583,SeniorCitizens,retired,married,unknown,no,LessThan10000,no,no,telephone,10,sep,LessThan365secs,2,2,success,yes
5584,MiddleAged,management,married,tertiary,no,LessThan10000,no,no,cellular,19,nov,LessThan730secs,2,0,unknown,yes
5585,Adults,management,married,tertiary,no,LessThan10000,no,no,cellular,11,jan,LessThan365secs,2,0,unknown,yes


In [237]:
print(len(train_data.index))

5581


In [238]:
def unique_vals(data,attr):
    """"find unique values for column in dataset"""
    col = data[attr].unique()
    return list(col)

def getClassCount(data,attr,classname):
    """counts no. of examples for each given class"""
    return list((data[attr].values)).count(classname)

def getProportion(data,attr,classname):
    """propotion of particular class value"""
    return getClassCount(data,attr,classname)/len(list((data[attr].values)))


def entropy(data,attr,classname,lcol):
    """
        this function returns entropy of particular
        atrribute : 'attr' with  labels stored in 'lcol'
        for particular class(value) as 'classname'
        --additional : pass attr=lcol to get entropy of system
    """
    pyes=0.0
    pno=0.0
    if attr == lcol:
        #this condition is to find the entropy of system
        pyes = getProportion(data,attr,'yes')
        pno =  getProportion(data,attr,'no')
        lgyes = 0.0
        lgno = 0.0
        if(pyes != 0.0):
            lgyes=math.log2(pyes)
        if(pno != 0.0):
            lgno=math.log2(pno)
        entrpy = -pyes*lgyes-pno*lgno;
        return entrpy
    #only consider a part of dataframe with value classname
    df = data[data[attr] == classname]
    n = len(list(df[attr].values))
    if(n==0):
        #no such attribute value the enntropy can not be determind
        print('Error : No such attribute as',classname)
        return -1.0
    pyes = len(list(df[df[lcol]=='yes'][attr].values))/n
    pno = len(list(df[df[lcol]=='no'][attr].values))/n
    
    lgyes = 0.0
    lgno= 0.0
    
    if(pyes != 0.0):
        lgyes=math.log2(pyes)
    if(pno != 0.0):
        lgno=math.log2(pno)
    #H(S) = -P1log2(P1)-P2log2(P2)
    entrpy = -pyes*lgyes-pno*lgno;
    return entrpy

def split(data,attr,classname):
    """split data for given classname of given attribute"""
    return data[data[attr]==classname]

def find_best_node(data):
    """
        returns the name and entropy of maximum entropy attribute for give 'data'
    """
    columns , lcol= list(data.columns)[:-1],list(data.columns)[-1]
    max_tot_entropy = -1.0
    best_attr = ""
    sys_entropy = entropy(data,lcol,'-',lcol);
    # print("{0:10}   {1:10}".format("attribute","entropy"))
    # print('-----------------------')  
    for col in columns:
        tot_entropy = 0.0
        for classname in unique_vals(data,col):
            tot_entropy += getProportion(data,col,classname)*entropy(data,col,classname,lcol)
        tot_entropy = sys_entropy-tot_entropy
        # print("{0:10}   {1:2f}".format(col,tot_entropy))
        if(max_tot_entropy<tot_entropy):
            max_tot_entropy = tot_entropy
            best_attr = col
    # print("**Best Choice :",best_attr)
    return best_attr,max_tot_entropy

In [239]:
print('unique vals outlook : ',unique_vals(getdata(),'outlook'))
print('number of elements with outlook as overcast are',getClassCount(getdata(),'outlook','overcast'))
print('proportion of overcast : ',getProportion(getdata(),'outlook','overcast'))
print("system's entropy :",entropy(getdata(),'play','temp','play'))
print('test split for sunny:')
print(split(getdata(),'outlook','sunny'))
print('H(sunny) : ',entropy(getdata(),'outlook','sunny','play'))
data = getdata()
print('(attr,gain) = ',find_best_node(data))

unique vals outlook :  ['sunny', 'overcast', 'rainy']
number of elements with outlook as overcast are 4
proportion of overcast :  0.2857142857142857
system's entropy : 0.9402859586706309
test split for sunny:
   outlook  temp humidity  windy play
0    sunny   hot     high  False   no
1    sunny   hot     high   True   no
7    sunny  mild     high  False   no
8    sunny  cool   normal  False  yes
10   sunny  mild   normal   True  yes
H(sunny) :  0.9709505944546686
(attr,gain) =  ('outlook', 0.2467498197744391)


In [240]:
class DecisionNode:
    """
        This Node Stores the attribute on which decision needs to be taken
        and further Sub-DecisionNodes
        data : dataframe of data on which this node was trained
        attr : decison attribute
        child : sub-decision Nodes 
    """
    def __init__(self,data,attr,child):
        self.data= data
        self.attr = attr
        self.childs = child


In [241]:
class Leaf:
    """
        Stores the prediction to return when its instance is encountered
    """
    def __init__(self,prediction):
        self.prediction  = prediction

In [242]:
class DecisionTree:

    """
        Decision Tree is the classifier class implementing ID3 Algorithm
        for decison making 
                                  
    """

    def __init__(self,data,lcol):
        """
            data :  pandas dataframe on which it will be trained
            lcol :  column in which labels are stored 
        """
        self.data=data
        self._root=None
        self.lcol = lcol
        pass
    
    def fit(self):
        """
            this is recursive fuction which trains 
                Decision tree and builds it simultaneously
        """
        self._root=self._fit(self.data,list(self.data.columns),self.lcol)

    def _fit(self,data,col_list,lcol):
        
        attr,gain = find_best_node(data)
        
        if(gain == 0 or len(unique_vals(data,lcol))==1 or len(col_list)==1):
            return Leaf(unique_vals(data,lcol)[0])
        
        branches = {}
        col_list.remove(attr)

        for classname in unique_vals(data,attr):
            next_data = split(data,attr,classname)
            branches[classname] = self._fit(next_data[col_list],col_list,lcol);
            
        return DecisionNode(data,attr,branches)
    
    def _tree_structure(self,root,lvl):
        if root == None:
            return
        if isinstance(root,DecisionNode):
            print('\t'*2*lvl,'attribute : ',root.attr)
        elif isinstance(root,Leaf):
            print('\t'*2*lvl,'prediction : ',root.prediction)
            return

        for classname in unique_vals(root.data,root.attr):
            try:
                print('\t'*2*lvl,'edge : ',classname)
                self._tree_structure(root.childs[classname],lvl+1)
            except : 
                pass

    def visualtize(self):
        """
            call this to visualize the tree structure
        """
        if self._root==None:
            print('Call fit() to create a tree first')
            return
        self._tree_structure(self._root,0)

    def predict_point(self,root,test_data_point):
        """
            predicts the label for single data_point
            and returns the result

            root : root of the decision tree
            test_data_point : data point for which result is to be predicted

        """
        prediction=''
        if(isinstance(root,Leaf)):
            prediction = root.prediction
        elif(isinstance(root,DecisionNode)):
            prediction=self.predict_point(root.childs[test_data_point[root.attr]],test_data_point)
        return prediction

    def predict(self,test_data,lcol):
        """
            test_data : data whose output needs to be predicted
            lcol      : column in which the lables are stored
            returns accurracy of predictions in precentage(%)
        """
        if(list(self.data.columns)!=list(test_data.columns)):
            #data passed to the for prediction is not made for this decison tree
            print('Error : Data Columns does not match')
            print('required :',list(self.data.columns))
            print('found :',list(test_data.columns))
            return None
        cols = list(test_data.columns)
        cols.remove(lcol)
        test_records = test_data[cols].to_dict(orient = 'records')
        test_labels = test_data[[lcol]].to_dict(orient = 'records')
        currect_cnt = 0
        tot_records = len(test_records)
        rec_no = 0 
        for record in test_records:
            try : 
                prediction = self.predict_point(self._root,record)
                if prediction == test_labels[rec_no][lcol]:
                    currect_cnt+=1
            except  KeyError:
                pass
            rec_no+=1
        
        return (currect_cnt/tot_records)*100

    def getRoot(self):
        return self._root; 

In [243]:
#test for wheather data set
data = getdata()
test_data=data
tree = DecisionTree(data,list(data.columns)[-1])
tree.fit()
tree.visualtize()
test_point = {
    'outlook':'rainy',
    'temp' : 'hot',
    'humidity' : 'high',
    'windy' : False,
}
print('data point prediction for',test_point,'is',tree.predict_point(tree.getRoot(),test_point))
print('Accurracy : ',tree.predict(test_data,list(test_data.columns)[-1]),'for given data')

 attribute :  outlook
 edge :  sunny
		 attribute :  humidity
		 edge :  high
				 prediction :  no
		 edge :  normal
				 prediction :  yes
 edge :  overcast
		 prediction :  yes
 edge :  rainy
		 attribute :  windy
		 edge :  False
				 prediction :  yes
		 edge :  True
				 prediction :  no
data point prediction for {'outlook': 'rainy', 'temp': 'hot', 'humidity': 'high', 'windy': False} is yes
Accurracy :  100.0 for given data


In [246]:
#test for bank data set
data,test_data = getBankData(0.75)
tree = DecisionTree(data,list(data.columns)[-1])
tree.fit()
tree.visualtize()
print('Accurracy : ',tree.predict(test_data,list(test_data.columns)[-1]))

 attribute :  duration
 edge :  LessThan365secs
		 attribute :  month
		 edge :  jun
				 attribute :  contact
				 edge :  unknown
						 attribute :  day
						 edge :  18
								 prediction :  no
						 edge :  17
								 attribute :  marital
								 edge :  divorced
										 prediction :  no
								 edge :  married
										 prediction :  no
								 edge :  single
										 attribute :  job
										 edge :  admin.
												 prediction :  no
										 edge :  technician
												 attribute :  education
												 edge :  tertiary
														 prediction :  no
												 edge :  secondary
														 prediction :  yes
										 edge :  services
												 prediction :  no
						 edge :  20
								 prediction :  no
						 edge :  11
								 prediction :  no
						 edge :  3
								 prediction :  no
						 edge :  16
								 prediction :  no
						 edge :  2
								 prediction :  no
						 edge :  9
								 prediction :  no
						 edge :  6
								 